In [112]:
import pandas as pd
import requests

In [113]:
year = '2023-01-01'
params = {
    'page[size]': '100',
    'page[number]': '1',
    'filter': f'record_date:gte:{year}'
}
endpoint = '/v1/accounting/dts/deposits_withdrawals_operating_cash'
base_url = 'https://api.fiscaldata.treasury.gov/services/api/fiscal_service'
url = base_url + endpoint
all_data = []

In [ ]:
class Data:
    def __init__(self, url: str, page_size: int, params=None) -> None:
        self.all_data = []
        self.url = url
        self.params = params
        self.page_size = page_size
        self.last_accessed = None
    
    def page_size(self):
        if self.params['page[size]'] == '100':
            prompt = input('Do you know the maximum page size that the API allows? (y/n) ')
            while prompt.lower() == 'y':
                try:
                    max_page_size = input('Enter the maximum page size (Hit enter to default 100): ')
                    if max_page_size == '':
                        break
                    else:
                        max_page_size = int(max_page_size)
                    self.params['page[size]'] = str(max_page_size)
                    prompt = 'n'
                except:
                    print('Invalid input. Try again.')
    
    def grab_data(self, all_data: list, url: str, params=None) -> pd.DataFrame:
        previous_response_status_code = None
        if params['page[size]'] == '100':
            prompt = input('Do you know the maximum page size that the API allows? (y/n) ')
            while prompt.lower() == 'y':
                try:
                    max_page_size = input('Enter the maximum page size (Hit enter to default 100): ')
                    if max_page_size == '':
                        break
                    else:
                        max_page_size = int(max_page_size)
                    params['page[size]'] = str(max_page_size)
                    prompt = 'n'
                except:
                    print('Invalid input. Try again.')
        
        while True:
            response = requests.get(url, params=params)
            
            # if previous_response_status_code == None:
            #     previous_response_status_code = response.status_code
            
            if response.status_code != 200 and previous_response_status_code != None:
                if previous_response_status_code == 200:
                    break
                else:
                    page_size = int(params['page[size]'])
                    params['page[size]'] = str(page_size // 2)
                    continue

            
            page_data = response.json().get('data', [])
            
            if not page_data:
                break
            
            all_data.extend(page_data)
            
            previous_response_status_code = response.status_code
            params['page[size]'] = str(int(params['page[size]']) * 10)
            params['page[number]'] = str(int(params['page[number]']) + 1)
        params['page[number]'] = '1'
        
        if all_data:
            df = pd.DataFrame(all_data)
            print(df.shape)
        else:
            print('No data found')

        print(params['page[size]'])
        return df

In [114]:
def grab_data(all_data: list, url: str, params=None) -> pd.DataFrame:
    previous_response_status_code = None
    if params['page[size]'] == '100':
        prompt = input('Do you know the maximum page size that the API allows? (y/n) ')
        while prompt.lower() == 'y':
            try:
                max_page_size = input('Enter the maximum page size (Hit enter to default 100): ')
                if max_page_size == '':
                    break
                else:
                    max_page_size = int(max_page_size)
                params['page[size]'] = str(max_page_size)
                prompt = 'n'
            except:
                print('Invalid input. Try again.')
    
    while True:
        response = requests.get(url, params=params)
        
        # if previous_response_status_code == None:
        #     previous_response_status_code = response.status_code
        
        if response.status_code != 200 and previous_response_status_code != None:
            if previous_response_status_code == 200:
                break
            else:
                page_size = int(params['page[size]'])
                params['page[size]'] = str(page_size // 2)
                continue

        
        page_data = response.json().get('data', [])
        
        if not page_data:
            break
        
        all_data.extend(page_data)
        
        previous_response_status_code = response.status_code
        params['page[size]'] = str(int(params['page[size]']) * 10)
        params['page[number]'] = str(int(params['page[number]']) + 1)
    params['page[number]'] = '1'
    
    if all_data:
        df = pd.DataFrame(all_data)
        print(df.shape)
    else:
        print('No data found')

    print(params['page[size]'])
    return df

def get_columns() -> list:
    pass

In [115]:
df = grab_data(all_data, url, params)

(11100, 17)
100000


In [116]:
df.columns

Index(['record_date', 'account_type', 'transaction_type', 'transaction_catg',
       'transaction_catg_desc', 'transaction_today_amt', 'transaction_mtd_amt',
       'transaction_fytd_amt', 'table_nbr', 'table_nm', 'src_line_nbr',
       'record_fiscal_year', 'record_fiscal_quarter', 'record_calendar_year',
       'record_calendar_quarter', 'record_calendar_month',
       'record_calendar_day'],
      dtype='object')

In [117]:
df['transaction_today_amt'] = pd.to_numeric(df['transaction_today_amt'])
df_sorted = df['transaction_today_amt'].sort_values(ascending=False)
print(df_sorted)

8903     410061
10950    377954
1447     370127
10949    366368
8801     345371
          ...  
6746        -17
10934       -32
4401        -50
10309       -88
68         -382
Name: transaction_today_amt, Length: 11100, dtype: int64


In [118]:
df.iloc[75428]

IndexError: single positional indexer is out-of-bounds